In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import warnings
from pylab import *

In [2]:
warnings.filterwarnings('ignore')
def extract_dt(df):
    df['time'] = pd.to_datetime(df['time'], format='%m%d %H:%M:%S')
    # df['month'] = df['time'].dt.month
    # df['day'] = df['time'].dt.day
    df['date'] = df['time'].dt.date
    df['hour'] = df['time'].dt.hour
    # df = df.drop_duplicates(['ship','month'])
    df['weekday'] = df['time'].dt.weekday
    return df

In [3]:
#cited from others
def group_feature(df, key, target, aggs):   
    agg_dict = {}
    for ag in aggs:
        agg_dict[f'{target}_{ag}'] = ag
    print(agg_dict)
    t = df.groupby(key)[target].agg(agg_dict).reset_index()
    return t

def extract_feature(df, train_feature):
    t = group_feature(df, 'ship','x',['max','min','mean','std','skew','sum'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    t = group_feature(df, 'ship','x',['count'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    t = group_feature(df, 'ship','y',['max','min','mean','std','skew','sum'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    t = group_feature(df, 'ship','v',['max','min','mean','std','skew','sum'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    t = group_feature(df, 'ship','d',['max','min','mean','std','skew','sum'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    train_feature['x_max_x_min'] = train_feature['x_max'] - train_feature['x_min']
    train_feature['y_max_y_min'] = train_feature['y_max'] - train_feature['y_min']
    train_feature['y_max_x_min'] = train_feature['y_max'] - train_feature['x_min']
    train_feature['x_max_y_min'] = train_feature['x_max'] - train_feature['y_min']
    train_feature['slope'] = train_feature['y_max_y_min'] / np.where(train_feature['x_max_x_min']==0, 0.001, train_feature['x_max_x_min'])
    train_feature['area'] = train_feature['x_max_x_min'] * train_feature['y_max_y_min']
    
    mode_hour = df.groupby('ship')['hour'].agg(lambda x:x.value_counts().index[0]).to_dict()
    train_feature['mode_hour'] = train_feature['ship'].map(mode_hour)
    
    t = group_feature(df, 'ship','hour',['max','min'])
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    
    hour_nunique = df.groupby('ship')['hour'].nunique().to_dict()
    date_nunique = df.groupby('ship')['date'].nunique().to_dict()
    train_feature['hour_nunique'] = train_feature['ship'].map(hour_nunique)
    train_feature['date_nunique'] = train_feature['ship'].map(date_nunique)

    t = df.groupby('ship')['time'].agg({'diff_time':lambda x:np.max(x)-np.min(x)}).reset_index()
    t['diff_day'] = t['diff_time'].dt.days
    t['diff_second'] = t['diff_time'].dt.seconds
    train_feature = pd.merge(train_feature, t, on='ship', how='left')
    return train_feature

In [4]:
#this function will get an n-2 length series
def translation_forward(list_in,n):
    for i in range(n):
        list_in.pop(0)
    return list_in

def area_tri(groupbied):
    # area_triangle=[]change to dataframe
    # 创建空DataFrame
    # ship_id=groupbied['ship'].unique().item(0)
    if len(groupbied) > 49:
        x1_list=list(groupbied['x'])
        y1_list=list(groupbied['y'])
        x2_list=list(translation_forward(x1_list,16))
        y2_list=list(translation_forward(y1_list,16))
        x3_list=list(translation_forward(x2_list,32))
        y3_list=list(translation_forward(y2_list,32))
        # t=list(groupbied['time'])
        delta_t1=list(groupbied['time'])
        delta_t2=list(translation_forward(delta_t1,48))
        area_list=[]
        line_2_list=[]
        eff_a_l_list=[]
        eff_a_t_list=[]
        for x1, y1, x2, y2, x3, y3, diff_t1, diff_t2 in zip(x1_list,y1_list,x2_list,y2_list,x3_list,y3_list,delta_t1,delta_t2):
            a=np.sqrt((x2-x3)**2+(y2-y3)**2)
            b=np.sqrt((x1-x3)**2+(y1-y3)**2)
            c=np.sqrt((x1-x2)**2+(y1-y2)**2)
            s = (a + b + c) / 2
            l=a+c
            area=(s*(s-a)*(s-b)*(s-c)) ** 0.5
            diff_t=(diff_t2 - diff_t1)/np.timedelta64(1, 'h')
            eff_area_line=area/l if l!=0 else 0
            eff_area_diff_t=area /diff_t if diff_t!=0 else 0
            # area_triangle.append(s)
            area_list.append(area)
            line_2_list.append(l)
            eff_a_l_list.append(eff_area_line)
            eff_a_t_list.append(eff_area_diff_t)
        area_triangle = pd.DataFrame(list(zip(area_list, line_2_list, eff_a_l_list, eff_a_t_list)))
        area_triangle.columns = ['area_3','Line_2','eff_a_l','eff_a_t']  
        # area_triangle = area_triangle.append(temp_df,ignore_index=True)
    else:
        area_triangle=pd.DataFrame({'area_3':[0],'Line_2':[0],'eff_a_l':[0],'eff_a_t':[0]})
    return area_triangle

In [5]:
#add FFT
def fft_v(groupbied):
    nSampleNum = len(groupbied)
    sampleTime = groupbied['diff_time'].mean()
    ncount = (nSampleNum-1)*sampleTime
    delta_f = nSampleNum / ncount
    x = np.linspace(0,sampleTime,ncount)#时域波形x轴坐标
    freqLine = nSampleNum
    v = groupbied['v'].values#np.sin(2*pi*250*x)
    fft = abs(np.fft.fft(v))[0:freqLine]  #调用fft变换算法计算频域波形
    fftx = np.linspace(0,delta_f*freqLine,freqLine)  #频域波形x轴坐标
    fft_low = pd.DataFrame(fft[fftx<freqLine*0.15]).describe().T.add_prefix('fft_low_')
    fft_mid_l = pd.DataFrame(fft[(fftx>=freqLine*0.2) & (fftx<freqLine*0.5)]).describe().T.add_prefix('fft_mid_l_')
    fft_mid_h = pd.DataFrame(fft[(fftx>=freqLine*0.5) & (fftx<freqLine*0.85)]).describe().T.add_prefix('fft_mid_h_')
    fft_high = pd.DataFrame(fft[fftx>=freqLine*0.85]).describe().T.add_prefix('fft_high_')
    FFT_=pd.concat([fft_low, fft_mid_l, fft_mid_h, fft_high], axis=1)
    return FFT_

In [26]:
def feature_version2(df,feature_label):
    df_order=df.groupby('ship').apply(lambda x: x.sort_values('time',ascending=True))
    df_order.reset_index(level=0,drop=True,inplace=True)# save the inner index number step#1
    df_order.index.name ='inner_rev_index'# save the inner index number step#2
    df_order.reset_index(drop=False,inplace=True)#add 0-N index
    df_order['inner_ord_index']=df_order.groupby('ship')['inner_rev_index'].rank(ascending=False)#use the former subIndex to Mark the t0-tN
    df_order['diff_time']=pd.DataFrame(df_order.groupby('ship').apply(lambda x:x['time'].diff()/np.timedelta64(1, 'h'))).reset_index()['time']
    df_order['diff_dist']=pd.DataFrame(df_order.groupby('ship').apply(lambda x:np.sqrt(x['x'].diff()**2+x['y'].diff()**2))).reset_index()[0]
    df_order['est_v']=pd.DataFrame(df_order.groupby('ship').apply(lambda x:x['diff_dist']/x['diff_time'])).reset_index()[0]
    df_order['est_diff_dist']=pd.DataFrame(df_order.groupby('ship').apply(lambda x:x['v']*x['diff_time']*1852)).reset_index()[0]
    v_feature=pd.DataFrame(df_order.groupby('ship').apply(fft_v)).reset_index(level=0,drop=False)#edited
    feature_label = pd.merge(feature_label, v_feature, on='ship', how='left')
    temp=pd.DataFrame(df_order[df_order['inner_ord_index']==1][['ship','x','y']]).reset_index(drop=True).rename(columns={'x':'start_x', 'y':'start_y'}, inplace = False)
    feature_label = pd.merge(feature_label, temp, on='ship', how='left')
    temp=pd.DataFrame(df_order[df_order['inner_rev_index']==0][['ship','x','y']]).reset_index(drop=True).rename(columns={'x':'end_x', 'y':'end_y'}, inplace = False)
    feature_label = pd.merge(feature_label, temp, on='ship', how='left')
    temp=pd.DataFrame(df_order.groupby('ship')['diff_dist'].agg({'dist':sum})).reset_index()
    feature_label = pd.merge(feature_label, temp, on='ship', how='left')
    return feature_label

In [7]:
train = pd.read_hdf('../input/train.h5')
test = pd.read_hdf('../input/test.h5')
train_label = train.drop_duplicates('ship')
type_map = dict(zip(train_label['type'].unique(), np.arange(3)))
# rev means a reversed map relation
type_map_rev = {v:k for k,v in type_map.items()} 
train_label['type'] = train_label['type'].map(type_map)
train = extract_dt(train)
test = extract_dt(test)

In [34]:
#add more information about distribution on dist and time
train_order=train.groupby('ship').apply(lambda x: x.sort_values('time',ascending=True))
train_order.reset_index(level=0,drop=True,inplace=True)# save the inner index number step#1
train_order.index.name ='inner_rev_index'# save the inner index number step#2
train_order.reset_index(drop=False,inplace=True)#add 0-N index
print(train_order.index)

RangeIndex(start=0, stop=2699638, step=1)


In [56]:
train_order['inner_ord_index']=train_order.groupby('ship')['inner_rev_index'].rank(ascending=False)
train_order





















,inner_rev_index,ship,x,y,v,d,time,type,date,hour,weekday,inner_ord_index
0,413,0,6.118352e+06,5.130672e+06,0.00,0,1900-11-07 12:09:28,拖网,1900-11-07,12,2,1.0
1,412,0,6.118352e+06,5.130672e+06,0.00,0,1900-11-07 12:18:30,拖网,1900-11-07,12,2,2.0
2,411,0,6.118352e+06,5.130672e+06,0.00,0,1900-11-07 12:28:32,拖网,1900-11-07,12,2,3.0
3,410,0,6.118352e+06,5.130672e+06,0.00,0,1900-11-07 12:38:32,拖网,1900-11-07,12,2,4.0
4,409,0,6.118352e+06,5.130672e+06,0.00,0,1900-11-07 12:48:30,拖网,1900-11-07,12,2,5.0
5,408,0,6.118352e+06,5.130672e+06,0.00,0,1900-11-07 12:58:30,拖网,1900-11-07,12,2,6.0
6,407,0,6.118352e+06,5.130672e+06,0.00,0,1900-11-07 13:08:30,拖网,1900-11-07,13,2,7.0
7,406,0,6.118352e+06,5.130672e+06,0.00,0,1900-11-07 13:18:32,拖网,1900-11-07,13,2,8.0
8,405,0,6.118352e+06,5.130672e+06,0.00,0,1900-11-07 13:40:00,拖网,1900-11-07,13,2,9.0
9,404,0,6.118352e+06,5.130672e+06,0.05,0,1900-11-07 13:48:30,拖网,1900-11-07,13,2,10.0


In [ ]:
train_order['inner_ord_index']=list(train.index)#use the former subIndex to Mark the t0-tN

In [47]:
temp_2=pd.DataFrame(train_order[train_order['inner_ord_index']==0][['ship','x','y']]).reset_index(level=0,drop=False).rename(columns={'x':'start_x', 'y':'start_y'}, inplace = True)
temp_2

In [48]:
train_order[train_order['ship']==17]

,inner_rev_index,ship,x,y,v,d,time,type,date,hour,weekday,inner_ord_index
6446,264,17,6.234304e+06,5.250504e+06,0.92,314,1900-11-21 12:03:05,拖网,1900-11-21,12,2,122
6447,263,17,6.234304e+06,5.250504e+06,0.32,0,1900-11-21 12:13:08,拖网,1900-11-21,12,2,123
6448,262,17,6.234304e+06,5.250504e+06,0.11,0,1900-11-21 12:23:09,拖网,1900-11-21,12,2,124
6449,261,17,6.234304e+06,5.250504e+06,0.32,0,1900-11-21 12:33:10,拖网,1900-11-21,12,2,125
6450,260,17,6.234304e+06,5.250504e+06,0.00,0,1900-11-21 12:43:11,拖网,1900-11-21,12,2,126
6451,259,17,6.234304e+06,5.250504e+06,0.11,0,1900-11-21 13:03:07,拖网,1900-11-21,13,2,127
6452,258,17,6.234304e+06,5.250504e+06,0.32,352,1900-11-21 13:13:11,拖网,1900-11-21,13,2,128
6453,257,17,6.234304e+06,5.250504e+06,0.11,0,1900-11-21 13:23:12,拖网,1900-11-21,13,2,129
6454,256,17,6.234304e+06,5.250504e+06,0.32,0,1900-11-21 13:53:11,拖网,1900-11-21,13,2,130
6455,255,17,6.234304e+06,5.250504e+06,0.00,0,1900-11-21 14:03:09,拖网,1900-11-21,14,2,131


In [ ]:
train_order['diff_time']=pd.DataFrame(train_order.groupby('ship').apply(lambda x:x['time'].diff()/np.timedelta64(1, 'h'))).reset_index()['time']
train_order['diff_dist']=pd.DataFrame(train_order.groupby('ship').apply(lambda x:np.sqrt(x['x'].diff()**2+x['y'].diff()**2))).reset_index()[0]
train_order['est_v']=pd.DataFrame(train_order.groupby('ship').apply(lambda x:x['diff_dist']/x['diff_time'])).reset_index()[0]
train_order['est_diff_dist']=pd.DataFrame(train_order.groupby('ship').apply(lambda x:x['v']*x['diff_time']*1852)).reset_index()[0]

In [25]:
v_feature=pd.DataFrame(train_order.groupby('ship')['diff_dist'].agg({'dist':sum})).reset_index()
v_feature

,dist
ship,
0,35968.022927
1,190031.884257
2,3384.780162
3,153492.071398
4,152911.229840
5,305144.132873
6,466781.350174
7,447907.286645
8,464740.254157


In [11]:
temp=v_feature
temp=temp.reset_index(level=0,drop=False)

In [12]:
temp

,ship,fft_low_count,fft_low_mean,fft_low_std,fft_low_min,fft_low_25%,fft_low_50%,fft_low_75%,fft_low_max,fft_mid_l_count,...,fft_mid_h_75%,fft_mid_h_max,fft_high_count,fft_high_mean,fft_high_std,fft_high_min,fft_high_25%,fft_high_50%,fft_high_75%,fft_high_max
0,0,18.0,83.887142,20.625293,47.175858,67.579416,88.184464,101.971765,110.11,18.0,...,13.395191,20.741462,360.0,11.629517,17.445930,0.359979,4.511088,7.045699,10.342505,107.015931
0,1,18.0,147.012768,153.170569,30.443463,74.020849,99.350370,146.822637,619.05,18.0,...,17.370267,26.003670,331.0,14.728231,33.473458,0.545854,5.235012,8.332708,12.514176,466.919493
0,2,18.0,57.564297,22.687140,29.253883,46.006162,55.918018,62.849070,138.67,18.0,...,59.624705,64.354406,179.0,51.045664,10.059779,27.399145,43.608261,51.484586,58.068341,73.207900
0,3,18.0,126.425416,118.286181,28.043566,50.211547,88.676866,154.085054,492.90,18.0,...,28.903377,35.314139,281.0,24.332056,29.207781,1.324192,11.032982,20.093767,26.913541,336.832017
0,4,18.0,123.666232,125.777353,7.531766,56.821201,91.643426,137.658744,566.30,18.0,...,37.834128,89.315933,347.0,32.401890,27.448425,1.809252,17.553343,25.929871,36.939714,262.391041
0,5,18.0,152.375111,186.085091,19.272286,49.139904,70.594554,133.495176,642.68,18.0,...,28.891941,34.058088,321.0,23.905038,41.160528,1.723069,11.805141,17.936838,23.478767,533.047879
0,6,18.0,103.670156,343.094106,3.943101,16.548537,21.650586,29.035464,1477.81,18.0,...,21.624657,33.631489,340.0,12.764592,7.182024,0.200111,7.437246,11.667549,16.700356,46.042178
0,7,15.0,269.208814,403.150860,36.177491,74.082905,126.441049,213.185323,1561.66,15.0,...,52.661721,62.156858,303.0,30.190478,53.769670,1.377165,13.030203,20.926804,32.271687,787.845823
0,8,18.0,94.020859,282.215158,8.456955,14.788280,26.157642,40.810715,1222.94,18.0,...,27.445733,44.944910,312.0,20.560298,11.760064,1.261237,11.461559,18.137247,28.883798,65.612300
0,9,18.0,0.949064,1.034886,0.251119,0.575736,0.678352,0.921383,4.97,18.0,...,0.654633,1.439506,343.0,0.473611,0.257234,0.022489,0.274523,0.474169,0.602427,1.439506


In [27]:
train_feature = extract_feature(train, train_label)
train_feature = feature_version2(train, train_feature)
# train_feature = pd.merge(train_feature, v_feature, on='ship', how='left')

train_feature.columns

{'x_max': 'max', 'x_min': 'min', 'x_mean': 'mean', 'x_std': 'std', 'x_skew': 'skew', 'x_sum': 'sum'}
{'x_count': 'count'}
{'y_max': 'max', 'y_min': 'min', 'y_mean': 'mean', 'y_std': 'std', 'y_skew': 'skew', 'y_sum': 'sum'}
{'v_max': 'max', 'v_min': 'min', 'v_mean': 'mean', 'v_std': 'std', 'v_skew': 'skew', 'v_sum': 'sum'}
{'d_max': 'max', 'd_min': 'min', 'd_mean': 'mean', 'd_std': 'std', 'd_skew': 'skew', 'd_sum': 'sum'}
{'hour_max': 'max', 'hour_min': 'min'}


Index(['ship', 'x', 'y', 'v', 'd', 'time', 'type', 'x_max', 'x_min', 'x_mean',
       'x_std', 'x_skew', 'x_sum', 'x_count', 'y_max', 'y_min', 'y_mean',
       'y_std', 'y_skew', 'y_sum', 'v_max', 'v_min', 'v_mean', 'v_std',
       'v_skew', 'v_sum', 'd_max', 'd_min', 'd_mean', 'd_std', 'd_skew',
       'd_sum', 'x_max_x_min', 'y_max_y_min', 'y_max_x_min', 'x_max_y_min',
       'slope', 'area', 'mode_hour', 'hour_max', 'hour_min', 'hour_nunique',
       'date_nunique', 'diff_time', 'diff_day', 'diff_second', 'fft_low_count',
       'fft_low_mean', 'fft_low_std', 'fft_low_min', 'fft_low_25%',
       'fft_low_50%', 'fft_low_75%', 'fft_low_max', 'fft_mid_l_count',
       'fft_mid_l_mean', 'fft_mid_l_std', 'fft_mid_l_min', 'fft_mid_l_25%',
       'fft_mid_l_50%', 'fft_mid_l_75%', 'fft_mid_l_max', 'fft_mid_h_count',
       'fft_mid_h_mean', 'fft_mid_h_std', 'fft_mid_h_min', 'fft_mid_h_25%',
       'fft_mid_h_50%', 'fft_mid_h_75%', 'fft_mid_h_max', 'fft_high_count',
       'fft_high_mean', '

In [20]:
# train_moved=train_order.loc[train_order.groupby('ship').apply(lambda x:x['diff_dist']!=0 ).reset_index()['diff_dist']]
# # df.loc[df.groupby(['CATEGORY'])['VALUE'].diff(1)!=0]This can drop off 连贯相同的项
# temp=pd.DataFrame(train_moved.groupby('ship').apply(area_tri)).reset_index()
# temp[temp['area_3']!=0]

Index(['ship', 'x', 'y', 'v', 'd', 'time', 'type', 'level_1', 'fft_low_count',
       'fft_low_mean', 'fft_low_std', 'fft_low_min', 'fft_low_25%',
       'fft_low_50%', 'fft_low_75%', 'fft_low_max', 'fft_mid_l_count',
       'fft_mid_l_mean', 'fft_mid_l_std', 'fft_mid_l_min', 'fft_mid_l_25%',
       'fft_mid_l_50%', 'fft_mid_l_75%', 'fft_mid_l_max', 'fft_mid_h_count',
       'fft_mid_h_mean', 'fft_mid_h_std', 'fft_mid_h_min', 'fft_mid_h_25%',
       'fft_mid_h_50%', 'fft_mid_h_75%', 'fft_mid_h_max', 'fft_high_count',
       'fft_high_mean', 'fft_high_std', 'fft_high_min', 'fft_high_25%',
       'fft_high_50%', 'fft_high_75%', 'fft_high_max', 'start_x', 'start_y',
       'end_x', 'end_y'],
      dtype='object')

In [15]:
# test_1=train_feature
temp_2=pd.DataFrame(train_order[train_order['inner_ord_index']==0][['ship','x','y']]).reset_index(drop=True).rename(columns={'x':'start_x', 'y':'start_y'}, inplace = False)
# test.index.name = 'ship'
# temp_2=temp_1[['ship','x']]
# temp_2
# temp_2.rename(columns={'x':'start_x', 'y':'start_y'}, inplace = False)
# temp_2
# test_1 = pd.merge(test_1, temp_2, on='ship', how='left')
# test_1
# train_feature

,ship,start_x,start_y
0,0,6.118352e+06,5.130672e+06
1,1,6.102450e+06,5.112760e+06
2,2,6.182482e+06,5.193696e+06
3,3,5.268205e+06,4.581391e+06
4,4,7.063407e+06,6.110886e+06
5,5,6.496477e+06,5.602987e+06
6,6,6.041016e+06,5.007689e+06
7,7,6.589755e+06,5.812012e+06
8,8,6.209616e+06,5.157619e+06
9,9,6.102853e+06,5.112531e+06


In [16]:
# t = np.arange(0, 1.0, 1.0/100)
# y = t+0.25
# a=pd.DataFrame(t[(y>0.5) & (y<0.6)])
# a.describe().add_prefix('prefix').T
# b=pd.concat([a, a], axis=1)
# b
# DataFrame
# temp1=(y>0.5)
# temp2=(y<0.6)
# temp3=temp1 and temp2
# temp3

In [17]:
# a=fft_v(train_order[train_order['ship']==0])
# a['ship']=1
# a